Gerekli Kütüphaneler

In [14]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time

# Makine öğrenmesi kütüphaneleri
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

# Değerlendirme metrikleri
from sklearn.metrics import mean_squared_error, r2_score

# Görselleştirme
import matplotlib.pyplot as plt
import seaborn as sns


Veri Çekme (Scraping) Fonksiyonu (Opsiyonel)

In [25]:
# Veri toplama
base_urls = [
    # Türkiye Süper Lig:
    "https://www.transfermarkt.com.tr/spieler-statistik/wertvollstespieler/marktwertetop?land_id=0&ausrichtung=alle&spielerposition_id=alle&altersklasse=alle&jahrgang=0&kontinent_id=0&plus=1",
    # İngiltere Premier Lig:
    "https://www.transfermarkt.com.tr/spieler-statistik/wertvollstespieler/marktwertetop?land_id=186&ausrichtung=alle&spielerposition_id=alle&altersklasse=alle&jahrgang=0&kontinent_id=0&plus=1",
    # İspanya La Liga:
    "https://www.transfermarkt.com.tr/spieler-statistik/wertvollstespieler/marktwertetop?land_id=40&ausrichtung=alle&spielerposition_id=alle&altersklasse=alle&jahrgang=0&kontinent_id=0&plus=1",
    # İtalya Serie A:
    "https://www.transfermarkt.com.tr/spieler-statistik/wertvollstespieler/marktwertetop?land_id=189&ausrichtung=alle&spielerposition_id=alle&altersklasse=alle&jahrgang=0&kontinent_id=0&plus=1",
    # Almanya Bundesliga:
    "https://www.transfermarkt.com.tr/spieler-statistik/wertvollstespieler/marktwertetop?land_id=74&ausrichtung=alle&spielerposition_id=alle&altersklasse=alle&jahrgang=0&kontinent_id=0&plus=1",
    # Fransa Ligue 1:
    "https://www.transfermarkt.com.tr/spieler-statistik/wertvollstespieler/marktwertetop?land_id=215&ausrichtung=alle&spielerposition_id=alle&altersklasse=alle&jahrgang=0&kontinent_id=0&plus=1",
    # Hollanda Eredivisie:
    "https://www.transfermarkt.com.tr/spieler-statistik/wertvollstespieler/marktwertetop?land_id=157&ausrichtung=alle&spielerposition_id=alle&altersklasse=alle&jahrgang=0&kontinent_id=0&plus=1"
]

print("Birden fazla lig için veri toplama başlıyor...")


"""
Birden fazla ligden oyuncu bilgilerini çeker ve bir CSV dosyasına kaydeder.
"""
headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:128.0) Gecko/20100101 Firefox/128.0'
}

response = requests.get(url, headers=headers)

page_number = 1
has_next_page = True
all_players_data = []

for base_url in base_urls:
    print(f"\nLig için veri toplanıyor: {base_url}")
    page_number = 1
    has_next_page = True

    while has_next_page:
        time.sleep(1)
        if page_number == 1:
            url = base_url
        else:
            url = f"{base_url}&page={page_number}"

        try:
            print(f"\nSayfa {page_number} işleniyor... URL: {url}")
            time.sleep(4)
            response = requests.get(url, headers=headers)
            if response.status_code == 404:
                print("Sayfa bulunamadı. Son sayfaya ulaşıldı.")
                break

            response.raise_for_status()
            soup = BeautifulSoup(response.content, 'html.parser')

            table = soup.find('table', class_='items')
            if not table:
                print("Tablo bulunamadı!")
                break

            rows = table.find_all('tr', class_=['odd', 'even'])
            if not rows:
                print("Bu sayfada oyuncu bulunamadı.")
                break

            print(f"Bu sayfada bulunan oyuncu sayısı: {len(rows)}")

            for row in rows:
                try:
                    cells = row.find_all('td')
                    # Tablo sütunlarından verileri al
                    rank = cells[0].text.strip() if len(cells) > 0 else "Unknown"
                    name = row.find('td', class_='hauptlink').text.strip()
                    position = cells[4].text.strip() if len(cells) > 4 else "Unknown"
                    age = cells[5].text.strip() if len(cells) > 5 else "Unknown"
                    nationality = cells[6].find('img')['title'] if cells[6].find('img') else "Unknown"
                    club = cells[7].find('a').find('img')['alt'] if cells[7].find('a') and cells[7].find('a').find('img') else cells[7].text.strip()
                    market_value = cells[8].text.strip() if len(cells) > 8 else "Unknown"

                    # İstatistik sütunları
                    matches = cells[9].text.strip() if len(cells) > 9 else "Unknown"
                    goals = cells[10].text.strip() if len(cells) > 10 else "Unknown"
                    own_goals = cells[11].text.strip() if len(cells) > 11 else "Unknown"
                    assists = cells[12].text.strip() if len(cells) > 12 else "Unknown"
                    yellow_cards = cells[13].text.strip() if len(cells) > 13 else "Unknown"
                    second_yellow = cells[14].text.strip() if len(cells) > 14 else "Unknown"
                    red_cards = cells[15].text.strip() if len(cells) > 15 else "Unknown"
                    substitutions = cells[16].text.strip() if len(cells) > 16 else "Unknown"

                    player_data = {
                        "Sıra No": rank,
                        "Oyuncu": name,
                        "Pozisyon": position,
                        "Yaş": age,
                        "Uyruk": nationality,
                        "Kulüp": club,
                        "Piyasa Değeri": market_value,
                        "Maç": matches,
                        "Gol": goals,
                        "Kendi Kalesine Gol": own_goals,
                        "Asist": assists,
                        "Sarı Kart": yellow_cards,
                        "İkinci Sarı Kart": second_yellow,
                        "Kırmızı Kart": red_cards,
                        "Oyuna Girdiği": substitutions
                    }

                    all_players_data.append(player_data)
                    print(f"Oyuncu eklendi: {name}")

                except Exception as e:
                    print(f"Oyuncu verisi işlenirken hata: {str(e)}")
                    continue

            next_page = soup.find('li', class_='tm-pagination__list-item tm-pagination__list-item--icon-next-page')
            if not next_page:
                print("Sonraki sayfa bulunamadı. Son sayfaya ulaşıldı.")
                has_next_page = False
            else:
                page_number += 1

            print(f"Şu ana kadar toplanan toplam oyuncu sayısı: {len(all_players_data)}")

        except requests.exceptions.RequestException as e:
            print(f"HTTP isteği hatası: {str(e)}")
            break
        except Exception as e:
            print(f"Beklenmeyen hata: {str(e)}")
            break

if not all_players_data:
    print("Hiç oyuncu verisi çekilemedi!")
    exit()

# Pandas DataFrame'e aktar ve CSV'ye kaydet
df = pd.DataFrame(all_players_data)
print("\nToplam Tablo:")
print(df.head())
df.to_csv('player_table_multiple_leagues.csv', index=False, encoding='utf-8')
print("\nVeriler 'player_table_multiple_leagues.csv' dosyasına kaydedildi.")



Birden fazla lig için veri toplama başlıyor...

Lig için veri toplanıyor: https://www.transfermarkt.com.tr/spieler-statistik/wertvollstespieler/marktwertetop?land_id=0&ausrichtung=alle&spielerposition_id=alle&altersklasse=alle&jahrgang=0&kontinent_id=0&plus=1

Sayfa 1 işleniyor... URL: https://www.transfermarkt.com.tr/spieler-statistik/wertvollstespieler/marktwertetop?land_id=0&ausrichtung=alle&spielerposition_id=alle&altersklasse=alle&jahrgang=0&kontinent_id=0&plus=1
Bu sayfada bulunan oyuncu sayısı: 25
Oyuncu eklendi: Erling Haaland
Oyuncu eklendi: Vinicius Junior
Oyuncu eklendi: Lamine Yamal
Oyuncu eklendi: Jude Bellingham
Oyuncu eklendi: Kylian Mbappé
Oyuncu eklendi: Bukayo Saka
Oyuncu eklendi: Florian Wirtz
Oyuncu eklendi: Jamal Musiala
Oyuncu eklendi: Phil Foden
Oyuncu eklendi: Cole Palmer
Oyuncu eklendi: Federico Valverde
Oyuncu eklendi: Rodri
Oyuncu eklendi: Declan Rice
Oyuncu eklendi: Martin Ødegaard
Oyuncu eklendi: Pedri
Oyuncu eklendi: Rodrygo
Oyuncu eklendi: Lautaro Martíne

Veri Temizleme ve Ön İşleme Fonksiyonları

In [26]:
def clean_market_value(value):
    """
    Piyasa değerini sayısal formata dönüştürür.
    Örnek: '200.00 mil. €' -> 200.0, '900 bin €' -> 0.9
    """
    if pd.isna(value):
        return np.nan
    value = value.lower().replace('€', '').strip()  # Küçük harfe çevir, '€' işaretini kaldır

    try:
        if 'mil.' in value:
            # 'mil. €' -> '200 mil.' gibi
            value = value.replace('mil.', '').strip()
            return float(value)
        elif 'bin' in value:
            # '900 bin' -> 0.9
            value = value.replace('bin', '').strip()
            return float(value) / 1000
        else:
            # Diğer durumlar
            return float(value)
    except ValueError:
        return np.nan  # Dönüşüm başarısız olursa NaN döndür

def categorize_positions(position):
    """
    Futbol pozisyonlarını ana kategorilere ayırır.
    """
    if pd.isna(position):
        return 'Diğer'

    position = position.lower()

    # Hücum pozisyonları
    if any(pos in position for pos in ['santrafor', 'forvet']):
        return 'Forvet'
    elif 'kanat' in position:
        return 'Kanat'
    elif 'on numara' in position:
        return 'Ofansif Orta Saha'

    # Orta saha pozisyonları
    elif any(pos in position for pos in ['merkez orta saha', 'ön libero']):
        return 'Merkez Orta Saha'
    elif 'defansif orta saha' in position:
        return 'Defansif Orta Saha'

    # Defans pozisyonları
    elif any(pos in position for pos in ['stoper', 'defans']):
        return 'Stoper'
    elif 'bek' in position:
        return 'Bek'

    # Kaleci
    elif 'kaleci' in position:
        return 'Kaleci'

    return 'Diğer'

def prepare_data(df):
    """
    Veri temizleme, dönüştürme ve model eğitimine uygun hale getirme adımlarını gerçekleştirir.
    """
    # Gerekli kolonları olduğunu varsayıyoruz
    df = df.copy()

    # 1. Tekrarlayan veya kullanışsız kolonları at
    columns_to_drop = ['Sıra No']  # 'Oyuncu', 'Kulüp' gibi metinsel analiz dışı kolonlar da atılabilir
    for col in columns_to_drop:
        if col in df.columns:
            df.drop(col, axis=1, inplace=True)

    # 2. Eksik veya hatalı değerleri NaN yap (örnek: '?')
    df.replace('?', np.nan, inplace=True)

    # 3. Veri tiplerini düzelt
    # Yaş, Maç, Gol vb. sayısal kolonları integer veya float'a çevirelim
    numeric_cols = ["Yaş", "Maç", "Gol", "Kendi Kalesine Gol", "Asist",
                    "Sarı Kart", "İkinci Sarı Kart", "Kırmızı Kart", "Oyuna Girdiği"]
    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    # Piyasa Değerini dönüştür
    df['Piyasa Değeri'] = df['Piyasa Değeri'].apply(clean_market_value)

    # 4. NaN değerleri barındıran satırları düşür (İsteğe bağlı: ortalama/medyan doldurma da kullanılabilir)
    df.dropna(inplace=True)

    # 5. Pozisyonları ana kategorilere dönüştür
    df['Pozisyon Kategori'] = df['Pozisyon'].apply(categorize_positions)

    # 6. Gerekirse "Oyuncu" ve "Kulüp" gibi metinsel verileri modelden çıkarabiliriz
    if 'Oyuncu' in df.columns:
        df.drop('Oyuncu', axis=1, inplace=True)
    if 'Kulüp' in df.columns:
        df.drop('Kulüp', axis=1, inplace=True)
    if 'Pozisyon' in df.columns:
        df.drop('Pozisyon', axis=1, inplace=True)

    # 7. Kategorik değişkenler: Uyruk ve Pozisyon Kategori
    # Label Encoding veya One-Hot Encoding
    label_encoders = {}
    cat_columns = ['Uyruk', 'Pozisyon Kategori']

    for cat_col in cat_columns:
        le = LabelEncoder()
        df[cat_col] = le.fit_transform(df[cat_col].astype(str))
        label_encoders[cat_col] = le

    # 8. Özellik (X) - Hedef (y) ayrımı
    y = df['Piyasa Değeri']
    X = df.drop(['Piyasa Değeri'], axis=1)

    # 9. Sayısal kolonları ölçeklendirme (StandardScaler)
    scaler = StandardScaler()

    # Kategorik kolonlar hariç diğer sayısal kolonları alalım
    # Pozisyon Kategori ve Uyruk da encode edilmiş olduğu için bunları da (istersek) ölçekleyebiliriz
    # ancak genelde label encoded değişkenler tam sayısal ölçeklendirme yerine
    # one-hot encoding ile kullanılır. Burada basitlik adına hepsini ölçekleyebiliriz.
    X_scaled = scaler.fit_transform(X)

    X = pd.DataFrame(X_scaled, columns=X.columns)

    return X, y, label_encoders, scaler


 Model Eğitim ve Değerlendirme Fonksiyonu

In [27]:
def train_and_evaluate_models(X, y, test_size=0.2, random_state=42):
    """
    Verilen X, y verisi üzerinde çeşitli modelleri (SVR, Decision Tree,
    Linear Regression, Random Forest) eğitir ve değerlendirir.
    """
    # Eğitim-test ayırma
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state
    )

    # Model sözlüğü
    models = {
        "Linear Regression": LinearRegression(),
        "Decision Tree": DecisionTreeRegressor(random_state=random_state),
        "Random Forest": RandomForestRegressor(n_estimators=100, random_state=random_state),
        "Support Vector Regressor (SVR)": SVR(kernel='rbf', C=1.0, epsilon=0.1)
    }

    results = {}
    for name, model in models.items():
        # Modeli eğit
        model.fit(X_train, y_train)

        # Tahmin
        y_pred = model.predict(X_test)

        # Metrikler
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        r2 = r2_score(y_test, y_pred)
        mae = np.mean(np.abs(y_test - y_pred))
        accuracy_percentage = r2 * 100  # R2'yi yüzdeye çevirelim

        results[name] = {
            'model': model,
            'MSE': mse,
            'RMSE': rmse,
            'MAE': mae,
            'R2': r2,
            'Accuracy (%)': accuracy_percentage
        }

        print(f"\n{name} Model Performansı:")
        print(f"  - MSE: {mse:.4f}")
        print(f"  - RMSE: {rmse:.4f}")
        print(f"  - MAE: {mae:.4f}")
        print(f"  - R2 Skoru: {r2:.4f}")
        print(f"  - Doğruluk (R2 %): %{accuracy_percentage:.2f}")

    return results


main

In [28]:
# 1) (Eğer veri çekmediyseniz) Veri çekme fonksiyonunu çağırın:
# scrape_transfermarkt_data('player_table_multiple_leagues.csv')

# 2) CSV'den veriyi oku
df = pd.read_csv('player_table_multiple_leagues.csv')

print("Ham veri ilk 5 satırı:")
display(df.head())

# 3) Veri ön işleme
X, y, label_encoders, scaler = prepare_data(df)

print("\nÖn işlenmiş X'in ilk 5 satırı:")
display(X.head())

print("\nHedef Değişken (y) ilk 5 değer:")
display(y.head())

# 4) Model Eğitim ve Değerlendirme
results = train_and_evaluate_models(X, y, test_size=0.2, random_state=42)

# 5) Sonuçların Karşılaştırılması
print("\n=== Model Karşılaştırma Sonuçları ===")
for model_name, metrics in results.items():
    print(f"{model_name} => MSE: {metrics['MSE']:.4f}, RMSE: {metrics['RMSE']:.4f}, R2: {metrics['R2']:.4f}, Accuracy(%): {metrics['Accuracy (%)']:.2f}")


Ham veri ilk 5 satırı:


,Sıra No,Oyuncu,Pozisyon,Yaş,Uyruk,Kulüp,Piyasa Değeri,Maç,Gol,Kendi Kalesine Gol,Asist,Sarı Kart,İkinci Sarı Kart,Kırmızı Kart,Oyuna Girdiği
0,1,Erling Haaland,Santrafor,24,Norveç,Manchester City,200.00 mil. €,35,29,0,2,2,0,0,0
1,2,Vinicius Junior,Sol Kanat,24,Brezilya,Real Madrid,200.00 mil. €,25,14,0,10,9,0,1,3
2,3,Lamine Yamal,Sağ Kanat,17,İspanya,FC Barcelona,180.00 mil. €,28,9,0,14,2,0,0,2
3,4,Jude Bellingham,On Numara,21,İngiltere,Real Madrid,180.00 mil. €,30,10,0,10,7,0,0,1
4,5,Kylian Mbappé,Santrafor,26,Fransa,Real Madrid,160.00 mil. €,31,17,0,3,2,0,0,3



Ön işlenmiş X'in ilk 5 satırı:


,Yaş,Uyruk,Maç,Gol,Kendi Kalesine Gol,Asist,Sarı Kart,İkinci Sarı Kart,Kırmızı Kart,Oyuna Girdiği,Pozisyon Kategori
0,-0.203595,0.372658,1.614216,5.076558,-0.163596,-0.168597,-0.272346,-0.224584,-0.201285,-1.073010,-0.967131
1,-0.203595,-1.053360,0.506171,2.077337,-0.163596,2.592487,2.923885,-0.224584,4.968080,-0.336971,-0.048763
2,-2.058934,1.085666,0.838584,1.077596,-0.163596,3.973029,-0.272346,-0.224584,-0.201285,-0.582317,-0.048763
3,-0.998740,0.943065,1.060193,1.277544,-0.163596,2.592487,2.010676,-0.224584,-0.201285,-0.827663,0.869605
4,0.326502,-0.625555,1.170998,2.677181,-0.163596,0.176538,-0.272346,-0.224584,-0.201285,-0.336971,-0.967131



Hedef Değişken (y) ilk 5 değer:


,Piyasa Değeri
0,200.0
1,200.0
2,180.0
3,180.0
4,160.0



Linear Regression Model Performansı:
  - MSE: 591.4228
  - RMSE: 24.3192
  - MAE: 18.0409
  - R2 Skoru: 0.4507
  - Doğruluk (R2 %): %45.07

Decision Tree Model Performansı:
  - MSE: 330.3930
  - RMSE: 18.1767
  - MAE: 9.4854
  - R2 Skoru: 0.6931
  - Doğruluk (R2 %): %69.31

Random Forest Model Performansı:
  - MSE: 293.9502
  - RMSE: 17.1450
  - MAE: 10.1378
  - R2 Skoru: 0.7270
  - Doğruluk (R2 %): %72.70

Support Vector Regressor (SVR) Model Performansı:
  - MSE: 850.0278
  - RMSE: 29.1552
  - MAE: 19.9525
  - R2 Skoru: 0.2105
  - Doğruluk (R2 %): %21.05

=== Model Karşılaştırma Sonuçları ===
Linear Regression => MSE: 591.4228, RMSE: 24.3192, R2: 0.4507, Accuracy(%): 45.07
Decision Tree => MSE: 330.3930, RMSE: 18.1767, R2: 0.6931, Accuracy(%): 69.31
Random Forest => MSE: 293.9502, RMSE: 17.1450, R2: 0.7270, Accuracy(%): 72.70
Support Vector Regressor (SVR) => MSE: 850.0278, RMSE: 29.1552, R2: 0.2105, Accuracy(%): 21.05
